# Student Information

Wangyang He


UIN 625004872


Nickname: heswaggy

Project submission 10

In [ ]:
import os
import numpy as np
import cv2
import json
import keras
from keras.models import load_model
from tqdm.notebook import tqdm

In [ ]:
!pip install --upgrade youtube_dl

     |████████████████████████████████| 1.9MB 13.2MB/s 


In [ ]:
test_videos = '/content/applyvideos'
os.mkdir(test_videos)

In [ ]:
import datetime
import youtube_dl, subprocess

#these are the videos link to test, uncomment to download them
urls = [
        'https://www.youtube.com/watch?v=73WPGgsfvt8',
        'https://www.youtube.com/watch?v=q_KdYNHtmZ8',
        'https://www.youtube.com/watch?v=iRLueS8j4zk',
        'https://www.youtube.com/watch?v=Ws6YiDt5Z9M',
        'https://www.youtube.com/watch?v=q7aH2ZWshHM',
        'https://www.youtube.com/watch?v=Kp9FOMbjFGE',
        'https://www.youtube.com/watch?v=Mq8Oe4fF4_I',
        'https://www.youtube.com/watch?v=Hhn04MJvEl4',
        'https://www.youtube.com/watch?v=jJhhSHgO5fk',
        'https://www.youtube.com/watch?v=mAqqU1VP5fc',
        'https://www.youtube.com/watch?v=5zhB-ym2gMU',
        'https://www.youtube.com/watch?v=k_Y5fM9lyBo',
        'https://www.youtube.com/watch?v=cPeGTpAavnk',
        'https://www.youtube.com/watch?v=9geYStBX1Iw',
        'https://www.youtube.com/watch?v=Mz7IbHsoNlk',
        'https://www.youtube.com/watch?v=TqA9a-5dO6Q',
        'https://www.youtube.com/watch?v=RLihptVh0SY'
         ]

for i, id in enumerate(urls):
  #save videos to the folder we created above
  true = os.path.join(test_videos,"video.{}.mp4".format(i))
  #download videos
  with youtube_dl.YoutubeDL({'format': 'best'}) as ydl:
    try:
      info = ydl.extract_info(id, download=False)
      video = info['entries'][0] if 'entries' in info else info
    except:
      continue
  #parameters
  url = video['url']
  starttime = datetime.timedelta(seconds=0)
  clip_length = datetime.timedelta(seconds=9999)
  subprocess.call(['ffmpeg', '-i', url, '-ss', str(starttime), '-t', str(clip_length), 
                   '-c:v', 'copy', '-c:a', 'copy', true,]) 
  print("finished downloading")

[youtube] 73WPGgsfvt8: Downloading webpage
[youtube] 73WPGgsfvt8: Downloading MPD manifest
finished downloading
[youtube] q_KdYNHtmZ8: Downloading webpage
finished downloading
[youtube] iRLueS8j4zk: Downloading webpage
[youtube] iRLueS8j4zk: Downloading MPD manifest
finished downloading
[youtube] Ws6YiDt5Z9M: Downloading webpage
finished downloading
[youtube] q7aH2ZWshHM: Downloading webpage
[youtube] q7aH2ZWshHM: Downloading MPD manifest
finished downloading
[youtube] Kp9FOMbjFGE: Downloading webpage
finished downloading
[youtube] Mq8Oe4fF4_I: Downloading webpage
[youtube] Mq8Oe4fF4_I: Downloading MPD manifest
finished downloading
[youtube] Hhn04MJvEl4: Downloading webpage
finished downloading
[youtube] jJhhSHgO5fk: Downloading webpage
[youtube] jJhhSHgO5fk: Downloading MPD manifest
finished downloading
[youtube] mAqqU1VP5fc: Downloading webpage
finished downloading
[youtube] 5zhB-ym2gMU: Downloading webpage
finished downloading
[youtube] k_Y5fM9lyBo: Downloading webpage
finished down

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/Submission 10/washingfeet_Xcep_sub10.h5') #this .h5 model file is also avaliable on my github.tamu repo
#loading trained model
print("model loaded")

model loaded


In [ ]:
def check_model(img, video, model): #this function will check the video with our model, and predict accruacy to produce JSON
    #get the video itself
    capture = cv2.VideoCapture(img)
    # get the video frame per second
    fps = capture.get(5) 
    # get all frames in the video
    video_frame = int(capture.get(7)) 
    # get video length in seconds
    length = int(video_frame/fps) 
    # get the number of predictions our model will make on the test video
    num_pred = int(length/2) 
    # frame per prediction
    fpp = video_frame // num_pred
    # length per frame
    dis = fpp // 15
    #count the frames for each 2 second clip
    count = 0
    pred_count = 1
    #some parameters to use in the for loop
    pred_time = []
    final_frame = []
    clips = []
    tmp = []

    for i in tqdm(range(video_frame)): #iterating over all the frames of the video
            #using openCV
            ret, frame = capture.read()
            count = count + 1

            if count == dis:
                count = count * 0
                frame=cv2.resize(frame,(150,150),interpolation=cv2.INTER_AREA)
                tmp.append(frame)
                final_frame.append(frame)
                
            if i == ((pred_count * fpp) - 1):
                inpt = np.array(tmp)
                inpt = inpt.astype('float32')
                inpt -= np.mean(inpt)
                inpt /= np.max(inpt)
                tmp = []
                data = np.expand_dims(inpt, axis=0)
                pred = model.predict(data, batch_size = 1) # verbose = 1
                rate = pred[0][1] #prediction score stored
                time_init = (pred_count - 1) / 0.5 #start time
                time_end = pred_count / 0.5  # end time
                #produce JSON file
                if rate > 0.8: #if prediction score better than 80%, add in the JSON file
                    time_result = ['Target: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                    json_format = {
                        'videoId': "RLihptVh0SY",
                        'type': "segment",
                        'startTime': np.float(time_init),
                        'endTime': np.float(time_end),
                        'observer': 'CSCE636Spring2021-heswaggy-10',
                        'isHuman': False,
                        'confirmedBySomeone': False,
                        'rejectedBySomeone': False,
                        'observation': {
                            'label': 'Washing feet',
                            'labelConfidence': np.float(rate)
                        }
                    }
                    clips.append(json_format)
                else:
                    time_result = ['Others: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                pred_count += 1
                pred_time.append(time_result)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    capture.release()
    cv2.destroyAllWindows()

    with open(video, 'w', encoding='utf-8') as f:
        json.dump(clips, f, indent=4)
    return final_frame

print("finished")

finished


In [ ]:
!mkdir ResultOfDetection

In [ ]:
final_frame = check_model('/content/applyvideos/video.16.mp4', '/content/ResultOfDetection/ResultOfDetection.16.json', model)

In [ ]:
!cp -a '/content/ResultOfDetection/' '/content/drive/MyDrive/Colab Notebooks/Submission 10'